In [15]:
import torch
import chess
import torch.nn as nn 
import torch.optim as optim 
import numpy as np
from torch.utils.data import DataLoader
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
from LMDBDataset import LMDBDataset

dataset = LMDBDataset("../../data/lmdb/")
loader = DataLoader(dataset, batch_size=128, shuffle=True, num_workers=0, pin_memory=True)

In [17]:
torch.set_printoptions(profile="full")

x, y = dataset[5]
print(x)

tensor([[[0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 0., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 1., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
       

In [18]:
print(y)

torch.set_printoptions(profile="default")

tensor(4505)


In [19]:
print("Ilość batchy:", len(loader))
print("ilość pozycji: ", len(loader)*loader.batch_size)

Ilość batchy: 12188
ilość pozycji:  1560064


In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
import torch
from torch import nn, optim
from model import ChessPolicyNet

model = ChessPolicyNet().to(device)

optimizer = optim.Adam(model.parameters(), lr=3e-4)
criterion = nn.CrossEntropyLoss()

scaler = torch.amp.GradScaler()

epochs = 12

for epoch in range(epochs):
    
    model.train()
    running_loss = 0.0

    for batch_idx, (batch_x, batch_y) in enumerate(loader):
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)

        optimizer.zero_grad()

        with torch.amp.autocast('cuda'):
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()

        if (batch_idx + 1) % 100 == 0:
            print(f"Epoch [{epoch+1}/{epochs}], Step [{batch_idx+1}/{len(loader)}], Loss: {running_loss/100:.4f}")
            running_loss = 0.0

    torch.save(model.state_dict(), f"chess_policy_epoch{epoch+1}_v3.pt")


Epoch [1/6], Step [100/12188], Loss: 6.5733
Epoch [1/6], Step [200/12188], Loss: 6.0363
Epoch [1/6], Step [300/12188], Loss: 5.9003
Epoch [1/6], Step [400/12188], Loss: 5.7840
Epoch [1/6], Step [500/12188], Loss: 5.6578
Epoch [1/6], Step [600/12188], Loss: 5.4864
Epoch [1/6], Step [700/12188], Loss: 5.2605
Epoch [1/6], Step [800/12188], Loss: 4.9730
Epoch [1/6], Step [900/12188], Loss: 4.7060
Epoch [1/6], Step [1000/12188], Loss: 4.4969
Epoch [1/6], Step [1100/12188], Loss: 4.2716
Epoch [1/6], Step [1200/12188], Loss: 4.0838
Epoch [1/6], Step [1300/12188], Loss: 3.9752
Epoch [1/6], Step [1400/12188], Loss: 3.8805
Epoch [1/6], Step [1500/12188], Loss: 3.7889
Epoch [1/6], Step [1600/12188], Loss: 3.7557
Epoch [1/6], Step [1700/12188], Loss: 3.6689
Epoch [1/6], Step [1800/12188], Loss: 3.6423
Epoch [1/6], Step [1900/12188], Loss: 3.5547
Epoch [1/6], Step [2000/12188], Loss: 3.5108
Epoch [1/6], Step [2100/12188], Loss: 3.5013
Epoch [1/6], Step [2200/12188], Loss: 3.4561
Epoch [1/6], Step [

```python
Network: chess_policy_epoch5.pt

Epoch [5/5], Step [6000/6094], Loss: 1.5345
```